In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, date

<h1>1. Compute deadhead matrix for three days<h1>


<h2> a) Retreive information of deadheads between locations <h2>

In [ ]:
information = pd.read_excel(r"/content/gdrive/My Drive/data logistics/DataCase1.xlsx", sheet_name="General")

# Make matrix of all deadhead times
# Select all rows in first sheet that contain information about deadhead times
information_deadhead = information.iloc[5:11,:]
# Reset index of rows
information_deadhead = information_deadhead.reset_index(drop=True)
# Name the index as Location so that it is clear what the index is about (in this case: Location A, Location B ect.)
information_deadhead.iloc[0,0] = "Location"
# Change the columns so that columns are also Location A, Location B, ect. such that we will have a matrix with the locations in the columns and rows and the values of deadhead in the matrix.
columns_deadhead = information_deadhead.iloc[0,:]
information_deadhead.columns = columns_deadhead
# Delete first row wich used to the names of the location (they are now set as the column names)
information_deadhead = information_deadhead.iloc[1:,:]
information_deadhead = information_deadhead.set_index('Location')

# Select all rows in first sheet that contain information about distances
kilometers_information = information.iloc[13:19,:]
# Name the index as Location so that it is clear what the index is about (in this case: Location A, Location B ect.)
kilometers_information.iloc[0,0] = "Location"
# Change the columns so that columns are also Location A, Location B, ect. such that we will have a matrix with the locations in the columns and rows and the values of kilometer in the matrix.
columns_deadhead = kilometers_information.iloc[0,:]
kilometers_information.columns = columns_deadhead
# Delete first row wich used to the names of the location (they are now set as the column names)
kilometers_information = kilometers_information.iloc[1:,:]
kilometers_information = kilometers_information.set_index('Location')


<ipython-input-3-8d275872aea1>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kilometers_information.iloc[0,0] = "Location"


<h1>2. Compute costs and compatible trips<h1>

<h2>a) Monday-Friday <h2>

In [ ]:
mofri_df = pd.read_excel(r"/content/gdrive/My Drive/data logistics/DataCase1.xlsx", sheet_name="Monday-Friday")

# Start creating cost matrix by iniating cost matrix
cost_mofri = pd.DataFrame(np.zeros((157, 157)))

compatible_trips = []

# Loop over every trip i and and create cost with every other existing trip j
for i in range(0,156):
  # Select the end trip time and end location of selected trip i
  end_time_trip = mofri_df.iloc[i,3]
  end_location_trip = str(mofri_df.iloc[i,1])

  for j in range (0,156):
    # Select the start time and start location of every respective paired trip j
    start_time_next = mofri_df.iloc[j,2]
    start_location_next = str(mofri_df.iloc[j,0])

    # Compute the time difference in hours and minutes between end time of trip i and start time of paired trip j
    time_diff = datetime.combine(date.today(), start_time_next) - datetime.combine(date.today(), end_time_trip)
    days, seconds = time_diff.days, time_diff.seconds
    hours = days * 24 + seconds // 3600
    minutes = (seconds % 3600) // 60

    # If the trips are compatible such that the start time and end time are within 50 minutes and there is no deadhead (i.e. location is the same) 
    if (hours >= 0) & (hours < 1) & (minutes <= 50) & (start_location_next == end_location_trip):
      cost = minutes * 0.5 + hours * 30
    # If the trips are not compatible assign very big cost such that these arcs will never be taken
    else: 
      cost = 1000000000

    cost_mofri.iloc[i,j] = cost


# Now assign all costs for the depot to every trip which will then be the start location
for d in range(0,156):
    # Select the start location of every trip d
    start_location_next = str(mofri_df.iloc[d,0])
    start_location = "Location " + start_location_next
    # Select how many km and deadhead to travel there is for that start location
    kilometers = kilometers_information[start_location]
    deadhead = information_deadhead[start_location]
    # Select the km and deadhead from depot to that start location (first row, i.e. row 0)
    kilometers_information_current = kilometers.iloc[0]
    deadhead_information_current = deadhead.iloc[0]

    # Assign cost with fixed costs of starting bus, wage and km cost
    cost = 2*kilometers_information_current + 0.5*deadhead_information_current + 100
    cost_mofri.iloc[156,d] = cost


# Now assign all costs from the every trip to the depot for finishing the route
for d in range(0,156):
    # Select the start location of every trip d
    end_location = str(mofri_df.iloc[d,1])
    end_location = "Location " + end_location
    # Select how many km and deadhead to travel there from end trip 
    kilometers = kilometers_information[end_location]
    deadhead = information_deadhead[end_location]

    # Select the km and deadhead from last trip to depot (first row, i.e. row 0)
    kilometers_information_current = kilometers.iloc[0]
    deadhead_information_current = deadhead.iloc[0]
    
    # Assign cost with fixed costs of starting bus, wage and km cost
    cost = 2*kilometers_information_current + 0.5*deadhead_information_current
    cost_mofri.iloc[d,156] = cost

compatible_trips_df = pd.DataFrame(compatible_trips, columns=['Start', 'End'])

compatible_trips_df.to_excel("/content/gdrive/My Drive/data logistics/Compatible_trips_MoFri.xlsx", sheet_name="MoFri")
cost_mofri.to_excel("/content/gdrive/My Drive/data logistics/cost_matrix_MoFri.xlsx", sheet_name="MoFri")



<h2>b) Saturday <h2>

In [ ]:
saturday_df = pd.read_excel(r"/content/gdrive/My Drive/data logistics/DataCase1.xlsx", sheet_name="Saturday")

length = len(saturday_df.index)

# Start creating cost matrix by iniating cost matrix
cost_mofri = pd.DataFrame(np.zeros((length+1, length+1)))


# Loop over every trip i and and create cost with every other existing trip j
for i in range(0,length):
  # Select the end trip time and end location of selected trip i
  end_time_trip = saturday_df.iloc[i,3]
  end_location_trip = str(saturday_df.iloc[i,1])

  for j in range (0,length):
    # Select the start time and start location of every respective paired trip j
    start_time_next = saturday_df.iloc[j,2]
    start_location_next = str(saturday_df.iloc[j,0])

    # Compute the time difference in hours and minutes between end time of trip i and start time of paired trip j
    time_diff = datetime.combine(date.today(), start_time_next) - datetime.combine(date.today(), end_time_trip)
    days, seconds = time_diff.days, time_diff.seconds
    hours = days * 24 + seconds // 3600
    minutes = (seconds % 3600) // 60

    # If the trips are compatible such that the start time and end time are within 50 minutes and there is no deadhead (i.e. location is the same) 
    if (hours >= 0) & (hours < 1) & (minutes <= 50) & (start_location_next == end_location_trip):
      cost = minutes * 0.5 + hours * 30
    # If the trips are not compatible assign very big cost such that these arcs will never be taken
    else: 
      cost = 1000000000

    cost_mofri.iloc[i,j] = cost


# Now assign all costs for the depot to every trip which will then be the start location
for d in range(0,length):
    # Select the start location of every trip d
    start_location_next = str(saturday_df.iloc[d,0])
    start_location = "Location " + start_location_next
    # Select how many km and deadhead to travel there is for that start location
    kilometers = kilometers_information[start_location]
    deadhead = information_deadhead[start_location]
    # Select the km and deadhead from depot to that start location (first row, i.e. row 0)
    kilometers_information_current = kilometers.iloc[0]
    deadhead_information_current = deadhead.iloc[0]

    # Assign cost with fixed costs of starting bus, wage and km cost
    cost = 2*kilometers_information_current + 0.5*deadhead_information_current + 100
    cost_mofri.iloc[length,d] = cost


# Now assign all costs from the every trip to the depot for finishing the route
for d in range(0,length):
    # Select the start location of every trip d
    end_location = str(saturday_df.iloc[d,1])
    end_location = "Location " + end_location
    # Select how many km and deadhead to travel there from end trip 
    kilometers = kilometers_information[end_location]
    deadhead = information_deadhead[end_location]

    # Select the km and deadhead from last trip to depot (first row, i.e. row 0)
    kilometers_information_current = kilometers.iloc[0]
    deadhead_information_current = deadhead.iloc[0]
    
    # Assign cost with fixed costs of starting bus, wage and km cost
    cost = 2*kilometers_information_current + 0.5*deadhead_information_current
    cost_mofri.iloc[d,length] = cost


cost_mofri.to_excel("/content/gdrive/My Drive/data logistics/cost_matrix_Sat.xlsx", sheet_name="Sat")

<h2> c) Sunday <h2>

In [ ]:
sunday_df = pd.read_excel(r"/content/gdrive/My Drive/data logistics/DataCase1.xlsx", sheet_name="Sunday")

length = len(sunday_df.index)

# Start creating cost matrix by iniating cost matrix
cost_sun = pd.DataFrame(np.zeros((length+1, length+1)))


# Loop over every trip i and and create cost with every other existing trip j
for i in range(0,length):
  # Select the end trip time and end location of selected trip i
  end_time_trip = sunday_df.iloc[i,3]
  end_location_trip = str(sunday_df.iloc[i,1])

  for j in range (0,length):
    # Select the start time and start location of every respective paired trip j
    start_time_next = sunday_df.iloc[j,2]
    start_location_next = str(sunday_df.iloc[j,0])

    # Compute the time difference in hours and minutes between end time of trip i and start time of paired trip j
    time_diff = datetime.combine(date.today(), start_time_next) - datetime.combine(date.today(), end_time_trip)
    days, seconds = time_diff.days, time_diff.seconds
    hours = days * 24 + seconds // 3600
    minutes = (seconds % 3600) // 60

    # If the trips are compatible such that the start time and end time are within 50 minutes and there is no deadhead (i.e. location is the same) 
    if (hours >= 0) & (hours < 1) & (minutes <= 50) & (start_location_next == end_location_trip):
      cost = minutes * 0.5 + hours * 30
    # If the trips are not compatible assign very big cost such that these arcs will never be taken
    else: 
      cost = 1000000000

    cost_sun.iloc[i,j] = cost


# Now assign all costs for the depot to every trip which will then be the start location
for d in range(0,length):
    # Select the start location of every trip d
    start_location_next = str(sunday_df.iloc[d,0])
    start_location = "Location " + start_location_next
    # Select how many km and deadhead to travel there is for that start location
    kilometers = kilometers_information[start_location]
    deadhead = information_deadhead[start_location]
    # Select the km and deadhead from depot to that start location (first row, i.e. row 0)
    kilometers_information_current = kilometers.iloc[0]
    deadhead_information_current = deadhead.iloc[0]

    # Assign cost with fixed costs of starting bus, wage and km cost
    cost = 2*kilometers_information_current + 0.5*deadhead_information_current + 100
    cost_sun.iloc[length,d] = cost


# Now assign all costs from the every trip to the depot for finishing the route
for d in range(0,length):
    # Select the start location of every trip d
    end_location = str(sunday_df.iloc[d,1])
    end_location = "Location " + end_location
    # Select how many km and deadhead to travel there from end trip 
    kilometers = kilometers_information[end_location]
    deadhead = information_deadhead[end_location]

    # Select the km and deadhead from last trip to depot (first row, i.e. row 0)
    kilometers_information_current = kilometers.iloc[0]
    deadhead_information_current = deadhead.iloc[0]
    
    # Assign cost with fixed costs of starting bus, wage and km cost
    cost = 2*kilometers_information_current + 0.5*deadhead_information_current
    cost_sun.iloc[d,length] = cost


cost_sun.to_excel("/content/gdrive/My Drive/data logistics/cost_matrix_Sun.xlsx", sheet_name="Sunday")